# ESG table scraper

In [2]:
!pip install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select

import time

In [4]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import urllib.request as req

from time import sleep

Chrome 브라우저를 제어하기 위한 WebDriver 객체를 생성하여 Chrome 브라우저를 자동으로 조작하고, Chrome 브라우저를 열어 주어진 URL로 이동하는 작업을 수행한다. 크롬 드라이버가 잘 작동하는지 확인하기 위해 아래와 같이 진행한다.

In [5]:
driver = webdriver.Chrome() # 경로 지정해서 넣기
driver.get('https://esg.krx.co.kr/contents/02/02020000/ESG02020000.jsp')

In [6]:
#2023년 esg등급 표 스크랩 함수
def esg_2023_scraper():
    driver = webdriver.Chrome()
    driver.get('https://esg.krx.co.kr/contents/02/02020000/ESG02020000.jsp')
    time.sleep(10) # 웹사이트가 켜지는 시간동안 잠시 멈춘다.
    
    global data_df_2023 #전역변수 설정 코드
    data_df_2023 = pd.DataFrame(columns=("회사명", "KCGS_종합등급", "KCGS_환경", "KCGS_사회", "KCGS_지배구조", "한국ESG연구소", "Moody's", "MSCI", "S&P", "연도"))
    
    index = 0

    while index < 794:
        table = driver.find_element(By.CLASS_NAME, 'esg_lank_tbl.mo_break_tbl')
        tbody = table.find_element(By.TAG_NAME, 'tbody')
        rows = tbody.find_elements(By.TAG_NAME, 'tr')
        
        # 한 페이지 속 기업들(rows)에 대해
        for i in range(len(rows)):
            # 매번 새로운 rows를 가져온다
            rows = tbody.find_elements(By.TAG_NAME, 'tr')
            row = rows[i]
            # 기업명
            company_name = row.find_element(By.TAG_NAME, 'a').text
            # KCGS_종합 등급 
            kcgs_1 = row.find_elements(By.CLASS_NAME, 'grd-font.kcgs.kcg')[0].text
            # KCGS_환경 등급
            kcgs_2 = row.find_elements(By.CLASS_NAME, 'grd-font.kcgs')[1].text
            # KCGS_사회 등급
            kcgs_3 = row.find_elements(By.CLASS_NAME, 'grd-font.kcgs')[2].text
            # KCGS_지배구조 등급
            kcgs_4 = row.find_elements(By.CLASS_NAME, 'grd-font.kcgs')[3].text
            #한국ESG연구소 등급
            kesg = row.find_elements(By.CLASS_NAME, 'grd-font.kesg')[0].text
            # Moody's 등급
            moodys = row.find_elements(By.CLASS_NAME, 'grd-font.moodys')[0].text
            # MSCI 등급
            msci = row.find_elements(By.CLASS_NAME, 'grd-font.msci')[0].text
            # S&P 등급
            sp = row.find_elements(By.CLASS_NAME, 'grd-font.sp')[0].text
            # 연도
            year = driver.find_elements(By.XPATH, '//*[@id="sch_yyc4ca4238a0b923820dcc509a6f75849b"]/option')[0].text
            
            # 기업별 추출한 정보를 dataframe의 한 행으로 저장
            data_df_2023.loc[index] = [company_name, kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, moodys, msci, sp, year]
            
            # index를 증가시키기 -> while이 반복되며 index가 794가 되면 종료
            index += 1
        
        # 한 페이지 내에는 10개의 row(기업) 존재. 10개 기업 데이터 모두 추출한 후에는 다음 페이지로 이동
        if index % 10 == 0:
            next_page = driver.find_element(By.XPATH, '//li[@class="next"]/a')
            driver.execute_script("arguments[0].click();", next_page)
            time.sleep(10)

    print(data_df_2023)
    return data_df_2023

In [7]:
#2022년 esg등급 표 스크랩 함수
def esg_2022_scraper():
    driver = webdriver.Chrome()
    driver.get('https://esg.krx.co.kr/contents/02/02020000/ESG02020000.jsp')
    time.sleep(10)
    
    #2022년 정보 조회
    select = Select(driver.find_element(By.XPATH, '//*[@name="sch_yy"]'))
    select.select_by_value("2022")
    driver.find_element(By.XPATH,'//*[@id="btnidc4ca4238a0b923820dcc509a6f75849b"]').click()
    time.sleep(10)
    
    global data_df_2022
    data_df_2022 = pd.DataFrame(columns=("회사명", "KCGS_종합등급", "KCGS_환경", "KCGS_사회", "KCGS_지배구조", "한국ESG연구소", "Moody's", "MSCI", "S&P", "연도"))
    
    index = 0

    while index < 794:
        table = driver.find_element(By.CLASS_NAME, 'esg_lank_tbl.mo_break_tbl')
        tbody = table.find_element(By.TAG_NAME, 'tbody')
        rows = tbody.find_elements(By.TAG_NAME, 'tr')
        
        # 한 페이지 속 기업들(rows)에 대해
        for i in range(len(rows)):
            # 매번 새로운 rows를 가져온다
            rows = tbody.find_elements(By.TAG_NAME, 'tr')
            row = rows[i]
            # 기업명
            company_name = row.find_element(By.TAG_NAME, 'a').text
            # KCGS_종합 등급 
            kcgs_1 = row.find_elements(By.CLASS_NAME, 'grd-font.kcgs.kcg')[0].text
            # KCGS_환경 등급
            kcgs_2 = row.find_elements(By.CLASS_NAME, 'grd-font.kcgs')[1].text
            # KCGS_사회 등급
            kcgs_3 = row.find_elements(By.CLASS_NAME, 'grd-font.kcgs')[2].text
            # KCGS_지배구조 등급
            kcgs_4 = row.find_elements(By.CLASS_NAME, 'grd-font.kcgs')[3].text
            #한국ESG연구소 등급
            kesg = row.find_elements(By.CLASS_NAME, 'grd-font.kesg')[0].text
            # Moody's 등급
            moodys = row.find_elements(By.CLASS_NAME, 'grd-font.moodys')[0].text
            # MSCI 등급
            msci = row.find_elements(By.CLASS_NAME, 'grd-font.msci')[0].text
            # S&P 등급
            sp = row.find_elements(By.CLASS_NAME, 'grd-font.sp')[0].text
            # 연도
            year = driver.find_elements(By.XPATH, '//*[@id="sch_yyc4ca4238a0b923820dcc509a6f75849b"]/option')[1].text
            
            # 기업별 추출한 정보를 dataframe의 한 행으로 저장
            data_df_2022.loc[index] = [company_name, kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, moodys, msci, sp, year]
            
            # index를 증가시키기 -> while이 반복되며 index가 794가 되면 종료
            index += 1
        
        # 한 페이지 내에는 10개의 row(기업) 존재. 10개 기업 데이터 모두 추출한 후에는 다음 페이지로 이동
        if index % 10 == 0:
            next_page = driver.find_element(By.XPATH, '//li[@class="next"]/a')
            driver.execute_script("arguments[0].click();", next_page)
            time.sleep(10)
    

    print(data_df_2022)
    return data_df_2022

In [8]:
#2021년 esg등급 표 스크랩 함수
def esg_2021_scraper():
    driver = webdriver.Chrome()
    driver.get('https://esg.krx.co.kr/contents/02/02020000/ESG02020000.jsp')
    time.sleep(10)
    
    #2021년 정보 조회
    select = Select(driver.find_element(By.XPATH, '//*[@name="sch_yy"]'))
    select.select_by_value("2021")
    driver.find_element(By.XPATH,'//*[@id="btnidc4ca4238a0b923820dcc509a6f75849b"]').click()
    time.sleep(10)
    
    global data_df_2021
    data_df_2021 = pd.DataFrame(columns=("회사명", "KCGS_종합등급", "KCGS_환경", "KCGS_사회", "KCGS_지배구조", "한국ESG연구소", "Moody's", "MSCI", "S&P", "연도"))
    
    index = 0

    while index < 794:
        table = driver.find_element(By.CLASS_NAME, 'esg_lank_tbl.mo_break_tbl')
        tbody = table.find_element(By.TAG_NAME, 'tbody')
        rows = tbody.find_elements(By.TAG_NAME, 'tr')
        
        # 한 페이지 속 기업들(rows)에 대해
        for i in range(len(rows)):
            # 매번 새로운 rows를 가져온다
            rows = tbody.find_elements(By.TAG_NAME, 'tr')
            row = rows[i]
            # 기업명
            company_name = row.find_element(By.TAG_NAME, 'a').text
            # KCGS_종합 등급 
            kcgs_1 = row.find_elements(By.CLASS_NAME, 'grd-font.kcgs.kcg')[0].text
            # KCGS_환경 등급
            kcgs_2 = row.find_elements(By.CLASS_NAME, 'grd-font.kcgs')[1].text
            # KCGS_사회 등급
            kcgs_3 = row.find_elements(By.CLASS_NAME, 'grd-font.kcgs')[2].text
            # KCGS_지배구조 등급
            kcgs_4 = row.find_elements(By.CLASS_NAME, 'grd-font.kcgs')[3].text
            #한국ESG연구소 등급
            kesg = row.find_elements(By.CLASS_NAME, 'grd-font.kesg')[0].text
            # Moody's 등급
            moodys = row.find_elements(By.CLASS_NAME, 'grd-font.moodys')[0].text
            # MSCI 등급
            msci = row.find_elements(By.CLASS_NAME, 'grd-font.msci')[0].text
            # S&P 등급
            sp = row.find_elements(By.CLASS_NAME, 'grd-font.sp')[0].text
            # 연도
            year = driver.find_elements(By.XPATH, '//*[@id="sch_yyc4ca4238a0b923820dcc509a6f75849b"]/option')[2].text
            
            # 기업별 추출한 정보를 dataframe의 한 행으로 저장
            data_df_2021.loc[index] = [company_name, kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, moodys, msci, sp, year]
            
            # index를 증가시키기 -> while이 반복되며 index가 794가 되면 종료
            index += 1
        
        # 한 페이지 내에는 10개의 row(기업) 존재. 10개 기업 데이터 모두 추출한 후에는 다음 페이지로 이동
        if index % 10 == 0:
            next_page = driver.find_element(By.XPATH, '//li[@class="next"]/a')
            driver.execute_script("arguments[0].click();", next_page)
            time.sleep(10)
    

    print(data_df_2021)
    return data_df_2021

In [9]:
#2020년 esg등급 표 스크랩 함수
def esg_2020_scraper():
    driver = webdriver.Chrome()
    driver.get('https://esg.krx.co.kr/contents/02/02020000/ESG02020000.jsp')
    time.sleep(10)
    
    #2020년 정보 조회
    select = Select(driver.find_element(By.XPATH, '//*[@name="sch_yy"]'))
    select.select_by_value("2020")
    driver.find_element(By.XPATH,'//*[@id="btnidc4ca4238a0b923820dcc509a6f75849b"]').click()
    time.sleep(10)
    
    global data_df_2020
    data_df_2020 = pd.DataFrame(columns=("회사명", "KCGS_종합등급", "KCGS_환경", "KCGS_사회", "KCGS_지배구조", "한국ESG연구소", "Moody's", "MSCI", "S&P", "연도"))
    
    index = 0

    while index < 794:
        table = driver.find_element(By.CLASS_NAME, 'esg_lank_tbl.mo_break_tbl')
        tbody = table.find_element(By.TAG_NAME, 'tbody')
        rows = tbody.find_elements(By.TAG_NAME, 'tr')
        
        # 한 페이지 속 기업들(rows)에 대해
        for i in range(len(rows)):
            # 매번 새로운 rows를 가져온다
            rows = tbody.find_elements(By.TAG_NAME, 'tr')
            row = rows[i]
            # 기업명
            company_name = row.find_element(By.TAG_NAME, 'a').text
            # KCGS_종합 등급 
            kcgs_1 = row.find_elements(By.CLASS_NAME, 'grd-font.kcgs.kcg')[0].text
            # KCGS_환경 등급
            kcgs_2 = row.find_elements(By.CLASS_NAME, 'grd-font.kcgs')[1].text
            # KCGS_사회 등급
            kcgs_3 = row.find_elements(By.CLASS_NAME, 'grd-font.kcgs')[2].text
            # KCGS_지배구조 등급
            kcgs_4 = row.find_elements(By.CLASS_NAME, 'grd-font.kcgs')[3].text
            #한국ESG연구소 등급
            kesg = row.find_elements(By.CLASS_NAME, 'grd-font.kesg')[0].text
            # Moody's 등급
            moodys = row.find_elements(By.CLASS_NAME, 'grd-font.moodys')[0].text
            # MSCI 등급
            msci = row.find_elements(By.CLASS_NAME, 'grd-font.msci')[0].text
            # S&P 등급
            sp = row.find_elements(By.CLASS_NAME, 'grd-font.sp')[0].text
            # 연도
            year = driver.find_elements(By.XPATH, '//*[@id="sch_yyc4ca4238a0b923820dcc509a6f75849b"]/option')[3].text
            
            # 기업별 추출한 정보를 dataframe의 한 행으로 저장
            data_df_2020.loc[index] = [company_name, kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, moodys, msci, sp, year]
            
            # index를 증가시키기 -> while이 반복되며 index가 794가 되면 종료
            index += 1
        
        # 한 페이지 내에는 10개의 row(기업) 존재. 10개 기업 데이터 모두 추출한 후에는 다음 페이지로 이동
        if index % 10 == 0:
            next_page = driver.find_element(By.XPATH, '//li[@class="next"]/a')
            driver.execute_script("arguments[0].click();", next_page)
            time.sleep(10)
    

    print(data_df_2020)
    return data_df_2020

In [10]:
esg_2023_scraper()

           회사명 KCGS_종합등급 KCGS_환경 KCGS_사회 KCGS_지배구조 한국ESG연구소 Moody's MSCI S&P  \
0       AJ네트웍스        B+       C      B+        B+        -       -    -   -   
1        AK홀딩스        B+       B       A         B        -       -    -   4   
2          BGF        B+       A      A+         B        -       -    -  18   
3       BGF리테일         A       A      A+         A        A      23  BBB  58   
4      BNK금융지주         A       A       A         A        A      37    A  43   
..         ...       ...     ...     ...       ...      ...     ...  ...  ..   
789  휴니드테크놀러지스         D       D       C         C        -       -    -   -   
790        휴비스        B+      B+       A        B+        -       -    -   6   
791        휴스틸         D       D       D         C        -       -    -   -   
792       흥국화재        B+       C      B+        B+        -       -    -   -   
793       흥아해운         C       C       B         C        -       -    -   -   

       연도  
0    2023  
1    2023  
2  

,회사명,KCGS_종합등급,KCGS_환경,KCGS_사회,KCGS_지배구조,한국ESG연구소,Moody's,MSCI,S&P,연도
0,AJ네트웍스,B+,C,B+,B+,-,-,-,-,2023
1,AK홀딩스,B+,B,A,B,-,-,-,4,2023
2,BGF,B+,A,A+,B,-,-,-,18,2023
3,BGF리테일,A,A,A+,A,A,23,BBB,58,2023
4,BNK금융지주,A,A,A,A,A,37,A,43,2023
...,...,...,...,...,...,...,...,...,...,...
789,휴니드테크놀러지스,D,D,C,C,-,-,-,-,2023
790,휴비스,B+,B+,A,B+,-,-,-,6,2023
791,휴스틸,D,D,D,C,-,-,-,-,2023
792,흥국화재,B+,C,B+,B+,-,-,-,-,2023


In [11]:
esg_2022_scraper()

           회사명 KCGS_종합등급 KCGS_환경 KCGS_사회 KCGS_지배구조 한국ESG연구소 Moody's MSCI S&P  \
0       AJ네트웍스        B+       C      B+        B+        -       -    -   -   
1        AK홀딩스        B+       B       A         B        -       -    -   4   
2          BGF        B+       A      A+         B        -       -    -  18   
3       BGF리테일         A       A      A+         A        A      23  BBB  58   
4      BNK금융지주         A       A       A         A        A      37    A  43   
..         ...       ...     ...     ...       ...      ...     ...  ...  ..   
789  휴니드테크놀러지스         D       D       C         C        -       -    -   -   
790        휴비스        B+      B+       A        B+        -       -    -   6   
791        휴스틸         D       D       D         C        -       -    -   -   
792       흥국화재        B+       C      B+        B+        -       -    -   -   
793       흥아해운         C       C       B         C        -       -    -   -   

       연도  
0    2022  
1    2022  
2  

,회사명,KCGS_종합등급,KCGS_환경,KCGS_사회,KCGS_지배구조,한국ESG연구소,Moody's,MSCI,S&P,연도
0,AJ네트웍스,B+,C,B+,B+,-,-,-,-,2022
1,AK홀딩스,B+,B,A,B,-,-,-,4,2022
2,BGF,B+,A,A+,B,-,-,-,18,2022
3,BGF리테일,A,A,A+,A,A,23,BBB,58,2022
4,BNK금융지주,A,A,A,A,A,37,A,43,2022
...,...,...,...,...,...,...,...,...,...,...
789,휴니드테크놀러지스,D,D,C,C,-,-,-,-,2022
790,휴비스,B+,B+,A,B+,-,-,-,6,2022
791,휴스틸,D,D,D,C,-,-,-,-,2022
792,흥국화재,B+,C,B+,B+,-,-,-,-,2022


In [12]:
esg_2021_scraper()

           회사명 KCGS_종합등급 KCGS_환경 KCGS_사회 KCGS_지배구조 한국ESG연구소 Moody's MSCI S&P  \
0       AJ네트웍스         B       D       B         B        -       -    -   -   
1        AK홀딩스        B+       B      B+        B+        -       -    -   1   
2          BGF         A       A      A+         A        -       -    -  13   
3       BGF리테일         A       A      A+         A        S       -  BBB  11   
4      BNK금융지주        A+       A      A+        A+        A       -    A  38   
..         ...       ...     ...     ...       ...      ...     ...  ...  ..   
789  휴니드테크놀러지스         B       C      B+        B+        -       -    -   -   
790        휴비스         A      B+      A+         A        -       -    -   5   
791        휴스틸         C       C       C         B        -       -    -   -   
792       흥국화재        B+       B       A         B        -       -    -   -   
793       흥아해운         D       D       C         D        -       -    -   -   

       연도  
0    2021  
1    2021  
2  

,회사명,KCGS_종합등급,KCGS_환경,KCGS_사회,KCGS_지배구조,한국ESG연구소,Moody's,MSCI,S&P,연도
0,AJ네트웍스,B,D,B,B,-,-,-,-,2021
1,AK홀딩스,B+,B,B+,B+,-,-,-,1,2021
2,BGF,A,A,A+,A,-,-,-,13,2021
3,BGF리테일,A,A,A+,A,S,-,BBB,11,2021
4,BNK금융지주,A+,A,A+,A+,A,-,A,38,2021
...,...,...,...,...,...,...,...,...,...,...
789,휴니드테크놀러지스,B,C,B+,B+,-,-,-,-,2021
790,휴비스,A,B+,A+,A,-,-,-,5,2021
791,휴스틸,C,C,C,B,-,-,-,-,2021
792,흥국화재,B+,B,A,B,-,-,-,-,2021


In [14]:
esg_2020_scraper()

           회사명 KCGS_종합등급 KCGS_환경 KCGS_사회 KCGS_지배구조 한국ESG연구소 Moody's MSCI S&P  \
0       AJ네트웍스         B       D       B         B        -       -    -   -   
1        AK홀딩스        B+       C      B+        B+        -       -    -   -   
2          BGF        B+       B       A        B+        -       -    -   -   
3       BGF리테일         A      B+      B+         A        -       -   BB   -   
4      BNK금융지주        A+      B+      A+        A+        -       -    A   -   
..         ...       ...     ...     ...       ...      ...     ...  ...  ..   
789  휴니드테크놀러지스         C       C       B         B        -       -    -   -   
790        휴비스        B+      B+      B+        B+        -       -    -   -   
791        휴스틸         C       B       B         C        -       -    -   -   
792       흥국화재         B       D       A         B        -       -    -   -   
793       흥아해운         D       D       C         D        -       -    -   -   

       연도  
0    2020  
1    2020  
2  

,회사명,KCGS_종합등급,KCGS_환경,KCGS_사회,KCGS_지배구조,한국ESG연구소,Moody's,MSCI,S&P,연도
0,AJ네트웍스,B,D,B,B,-,-,-,-,2020
1,AK홀딩스,B+,C,B+,B+,-,-,-,-,2020
2,BGF,B+,B,A,B+,-,-,-,-,2020
3,BGF리테일,A,B+,B+,A,-,-,BB,-,2020
4,BNK금융지주,A+,B+,A+,A+,-,-,A,-,2020
...,...,...,...,...,...,...,...,...,...,...
789,휴니드테크놀러지스,C,C,B,B,-,-,-,-,2020
790,휴비스,B+,B+,B+,B+,-,-,-,-,2020
791,휴스틸,C,B,B,C,-,-,-,-,2020
792,흥국화재,B,D,A,B,-,-,-,-,2020


In [15]:
data_df_2023.to_csv('esg_2023.csv')
data_df_2022.to_csv('esg_2022.csv')
data_df_2021.to_csv('esg_2021.csv')
data_df_2020.to_csv('esg_2020.csv')

### 데이터 합치기

In [16]:
data_df_merge = pd.concat([data_df_2023, data_df_2022, data_df_2021, data_df_2020])
data_df_merge

,회사명,KCGS_종합등급,KCGS_환경,KCGS_사회,KCGS_지배구조,한국ESG연구소,Moody's,MSCI,S&P,연도
0,AJ네트웍스,B+,C,B+,B+,-,-,-,-,2023
1,AK홀딩스,B+,B,A,B,-,-,-,4,2023
2,BGF,B+,A,A+,B,-,-,-,18,2023
3,BGF리테일,A,A,A+,A,A,23,BBB,58,2023
4,BNK금융지주,A,A,A,A,A,37,A,43,2023
...,...,...,...,...,...,...,...,...,...,...
789,휴니드테크놀러지스,C,C,B,B,-,-,-,-,2020
790,휴비스,B+,B+,B+,B+,-,-,-,-,2020
791,휴스틸,C,B,B,C,-,-,-,-,2020
792,흥국화재,B,D,A,B,-,-,-,-,2020


### 데이터 정렬

In [17]:
data_df_merge.sort_values(by=['회사명', '연도'], inplace = True)
data_df_merge

,회사명,KCGS_종합등급,KCGS_환경,KCGS_사회,KCGS_지배구조,한국ESG연구소,Moody's,MSCI,S&P,연도
0,AJ네트웍스,B,D,B,B,-,-,-,-,2020
0,AJ네트웍스,B,D,B,B,-,-,-,-,2021
0,AJ네트웍스,B+,C,B+,B+,-,-,-,-,2022
0,AJ네트웍스,B+,C,B+,B+,-,-,-,-,2023
1,AK홀딩스,B+,C,B+,B+,-,-,-,-,2020
...,...,...,...,...,...,...,...,...,...,...
792,흥국화재,B+,C,B+,B+,-,-,-,-,2023
793,흥아해운,D,D,C,D,-,-,-,-,2020
793,흥아해운,D,D,C,D,-,-,-,-,2021
793,흥아해운,C,C,B,C,-,-,-,-,2022


### 저장하기

In [18]:
data_df_merge.to_csv('esg_merge.csv', encoding='utf-8-sig')